In [1]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from glob import glob

from keras.preprocessing import image
import os
import matplotlib.pyplot as plt
import cv2

In [2]:
img_dims = 64
batch_size = 32

In [3]:
os.listdir('/kaggle/input/')

['chest-xray-pneumonia']

In [4]:
classifier = Sequential()
classifier.add(Conv2D(32, (3, 3), input_shape = (img_dims, img_dims, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Flatten())
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

2022-11-06 16:11:39.817826: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-06 16:11:39.827971: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-06 16:11:39.828708: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-06 16:11:39.829757: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [5]:

input_path = '../input/chest-xray-pneumonia/chest_xray/'
train_datagen = ImageDataGenerator(rescale = 1./255,
shear_range = 0.2,
zoom_range = 0.2,
horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory(directory=input_path+'train',
target_size = (img_dims, img_dims),
batch_size = batch_size,
class_mode = 'binary')
test_set = test_datagen.flow_from_directory(directory=input_path+'test',
target_size = (img_dims, img_dims),
batch_size = batch_size,
class_mode = 'binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [6]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [7]:
epochs = 10

hist = classifier.fit_generator(
           training_set, steps_per_epoch=training_set.samples // batch_size, 
           epochs=epochs, validation_data=test_set, 
           validation_steps= test_set.samples)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-11-06 16:11:51.293444: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


2022-11-06 16:11:53.094558: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


163/163 [==============================] - 96s 549ms/step - loss: 0.5020 - accuracy: 0.8056 - val_loss: 0.3389 - val_accuracy: 0.8317
Epoch 2/10
163/163 [==============================] - 59s 361ms/step - loss: 0.2148 - accuracy: 0.9109
Epoch 3/10
163/163 [==============================] - 58s 357ms/step - loss: 0.1955 - accuracy: 0.9193
Epoch 4/10
163/163 [==============================] - 58s 357ms/step - loss: 0.1921 - accuracy: 0.9227
Epoch 5/10
163/163 [==============================] - 61s 374ms/step - loss: 0.1687 - accuracy: 0.9339
Epoch 6/10
163/163 [==============================] - 62s 378ms/step - loss: 0.1598 - accuracy: 0.9371
Epoch 7/10
163/163 [==============================] - 61s 375ms/step - loss: 0.1547 - accuracy: 0.9394
Epoch 8/10
163/163 [==============================] - 62s 379ms/step - loss: 0.1565 - accuracy: 0.9375
Epoch 9/10
163/163 [==============================] - 59s 361ms/step - loss: 0.1554 - accuracy: 0.9394
Epoch 10/10
163/163 [=====================

In [15]:
img = plt.imread('../input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA/person1954_bacteria_4886.jpeg')
img = cv2.resize(img, (img_dims, img_dims))
img = np.dstack([img, img, img])
img = img.astype('float32') / 255
result = classifier.predict(np.expand_dims(image.img_to_array(img), axis = 0))  

if result[0][0] > 0.5:
  prediction = 'Pnuemonia'
else:
  prediction = 'Normal'
  
print(prediction)

Pnuemonia


In [16]:
classifier.save('pnuemonia_89.74_CNN_model.h5')